In [0]:
!pip install -q langdetect
!pip install autocorrect
import pandas as pd
import numpy as np
import re
import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
import time
import warnings
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from autocorrect import Speller
import string
from bs4 import BeautifulSoup
from langdetect import detect
from textblob import TextBlob

porter = PorterStemmer()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
warnings.filterwarnings(action = 'ignore') 

In [0]:
las_vegas_data= pd.read_pickle('/content/drive/My Drive/las_vegas_data.pkl')
las_vegas_data.shape

In [7]:
start= time.time()
las_vegas_data['length'] = las_vegas_data['text'].apply(len)
las_vegas_data=las_vegas_data[las_vegas_data['length']>=50]
las_vegas_data['language']=las_vegas_data['text'].apply(detect)
las_vegas_data=las_vegas_data[las_vegas_data['language']== 'en']
print("Time to load data: {} seconds".format(time.time() - start))

Time to load data: 8.343015909194946 seconds


In [0]:
las_vegas_data.to_pickle('/content/drive/My Drive/las_vegas_data_only_english.pkl')
las_vegas_data=pd.read_pickle('/content/drive/My Drive/las_vegas_data_only_english.pkl')

In [0]:
def filter_punctuation(text):
  filtered= "".join([c for c in text if c not in string.punctuation])
  return filtered
def filter_html(text):
  soup=BeautifulSoup(text, 'lxml')
  filtered= soup.get_text()
  return filtered
def filter_stopwords(text):
  stop_words = set(stopwords.words('english'))
  filtered= [item for item in text if item not in stop_words]
  return filtered
def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  lemmatized= [lemmatizer.lemmatize(i) for i in text]
  return lemmatized

def stemmize(text):
  stemmer = PorterStemmer()
  stemmed= " ".join([stemmer.stem(i) for i in text])
  return stemmed

In [0]:
start= time.time()
las_vegas_data.text = las_vegas_data.text.str.lower()
las_vegas_data['text']= las_vegas_data.text.str.replace('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')
las_vegas_data['text']=las_vegas_data['text'].apply(lambda x: filter_punctuation(x))
las_vegas_data['text']=las_vegas_data['text'].apply(lambda x: filter_html(x))
las_vegas_data['text']=las_vegas_data['text'].apply(lambda x: lower(x))
las_vegas_data['tokenized_text'] = las_vegas_data['text'].apply(sent_tokenize)
las_vegas_data['tokenized_text'] = las_vegas_data['text'].apply(word_tokenize) 
las_vegas_data['tokenized_text'] = las_vegas_data['tokenized_text'].apply(lambda tokens: [word for word in tokens if word.isalpha()])        
las_vegas_data['tokenized_text']=las_vegas_data['tokenized_text'].apply(lambda x: filter_stopwords(x))
las_vegas_data['tokenized_text']=las_vegas_data['tokenized_text'].apply(lambda x: lemmatize(x) )
# las_vegas_data['tokenized_text']=las_vegas_data['tokenized_text'].apply(lambda x: stemmer(x) )
print("Time to load data: {} seconds".format(time.time() - start))
las_vegas_data

In [0]:
# spell= Speller(lang='en')
# las_vegas_data['text']=las_vegas_data['text'].apply(spell)

In [27]:
start= time.time()
las_vegas_data['label'] = ''
las_vegas_data.loc[las_vegas_data.stars >=3, 'label'] = 'positive'
las_vegas_data.loc[las_vegas_data.stars <3, 'label'] = 'negative'
# las_vegas_data.tokenized_text = las_vegas_data.tokenized_text.str.lower()
print("Time to load data: {} seconds".format(time.time() - start))

Time to load data: 0.3048112392425537 seconds


In [0]:
las_vegas_data.drop(columns=['length'], inplace=True)
las_vegas_data.drop(columns=['language'], inplace=True)

In [0]:
las_vegas_data['polarity'] = las_vegas_data['text'].apply(lambda text: TextBlob(text).sentiment.polarity) 

In [0]:
las_vegas_data.to_pickle('/content/drive/My Drive/las_vegas_data_tokenized.pkl')

In [34]:
las_vegas_data

,business_id,review_id,stars,text,tokenized_text,label,polarity
0,tstimHoMcYbkSC4eBA1wEg,iWFBGYotfzwiLsOka0e1Rw,4.5,we found out about this gem from the mans cowo...,"[found, gem, man, coworker, used, live, apt, c...",positive,0.238242
1,tstimHoMcYbkSC4eBA1wEg,QhiVo4F8MFSoz1T7YggcXA,4.5,the chips are a fried corn tortilla which were...,"[chip, fried, corn, tortilla, surprisingly, am...",positive,0.342857
2,tstimHoMcYbkSC4eBA1wEg,FUv-c5CkLy71yFTGLrp2ag,4.5,the pollo mole is well worth the trip out here...,"[pollo, mole, well, worth, trip, service, frie...",positive,0.377083
3,tstimHoMcYbkSC4eBA1wEg,lJfGtkCc4in6Xcq-_Wtfwg,4.5,ive ordered caldo de rez from here twice and b...,"[ive, ordered, caldo, de, rez, twice, time, fo...",positive,0.109524
4,tstimHoMcYbkSC4eBA1wEg,BKoH9aG1ZjUjy2brzkxzJw,4.5,my boyfriend doesnt like mexican food yet he l...,"[boyfriend, doesnt, like, mexican, food, yet, ...",positive,0.245631
...,...,...,...,...,...,...,...
712589,p5rpYtxS5xPQjt3MXYVEwA,M5Z6ju1IxgXyHbuTB4b0Ew,4.0,weak to say the least nobody knows whats going...,"[weak, say, least, nobody, know, whats, going,...",positive,-0.229167
712590,p5rpYtxS5xPQjt3MXYVEwA,pn5TS8GiEs0iE_O1dTkWuQ,4.0,i ordered the chicken quinoa salad i cant beli...,"[ordered, chicken, quinoa, salad, cant, believ...",positive,0.257778
712591,p5rpYtxS5xPQjt3MXYVEwA,ACUg1ScLb6pwL6jJyp65-w,4.0,i love this place salmon with steamed asparagu...,"[love, place, salmon, steamed, asparagus, brow...",positive,0.471875
712592,p5rpYtxS5xPQjt3MXYVEwA,_51tDhnA8WJY8VmUg1Cu6w,4.0,this is my go to spot when im too lazy to find...,"[go, spot, im, lazy, find, food, food, smoothy...",positive,0.173333


In [0]:
# start= time.time()

# word_list=["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above",
# "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af",
# "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al",
# "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst",
# "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything",
# "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate",
# "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking",
# "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3",
# "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand",
# "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best",
# "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief",
# "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant",
# "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj",
# "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider",
# "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course",
# "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc",
# "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't",
# "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards",
# "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef",
# "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en",
# "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu",
# "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example",
# "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire",
# "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly",
# "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g",
# "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes",
# "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens",
# "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello",
# "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself",
# "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit",
# "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib",
# "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate",
# "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated",
# "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq",
# "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j",
# "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows",
# "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les",
# "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo",
# "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many",
# "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill",
# "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt",
# "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd",
# "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never",
# "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none",
# "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns",
# "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi",
# "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or",
# "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over",
# "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part",
# "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl",
# "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr",
# "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps",
# "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd",
# "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards",
# "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh",
# "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said",
# "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing",
# "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven",
# "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn",
# "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly",
# "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some",
# "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon",
# "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop",
# "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system",
# "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th",
# "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them",
# "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll",
# "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd",
# "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly",
# "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti",
# "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr",
# "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u",
# "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto",
# "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually",
# "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype",
# "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd",
# "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what",
# "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas",
# "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither",
# "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi",
# "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words",
# "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn",
# "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre",
# "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",'A', 'AA', 'AB', 'AC',
# 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AP', 'AR', 'AS', 'AT', 'AU','AV', 'AW', 'AX', 'AY',
# 'AZ', 'Aa', 'Ab', 'Ac', 'Ad', 'Ae', 'Af', 'Ah', 'Ai', 'Aj', 'Al', 'Am', 'An', 'Ap', 'Ar', 'As', 'At', 'Au', 'Av', 'Aw',
# 'Ax', 'Ay', 'Az', 'B', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BO', 'BP', 'BQ',
# 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'BZ', 'Ba', 'Bc', 'Be', 'Bf', 'Bg', 'Bi', 'Bj', 'Bk', 'Bl', 'Bo', 'Br',
# 'Bs', 'Bt', 'Bu', 'By', 'Bì', 'Bò', 'C', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'CM',
# 'CN', 'CO', 'CP', 'CR', 'CS', 'CT', 'CV', 'CW', 'CZ', 'Ca', 'Cc', 'Cd', 'Ce', 'Ch', 'Ci', 'Cj', 'Ck', 'Cm', 'Co', 'Cs',
# 'Ct', 'Cu', 'Cv', 'Cy', 'Cz', 'Cá', 'D', 'DA', 'DB', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DI', 'DJ', 'DK', 'DL', 'DM',
# 'DN', 'DO', 'DP', 'DQ', 'DR', 'DS', 'DT', 'DU', 'DV', 'DW', 'DX', 'DY', 'DZ', 'Da', 'Db', 'Dc', 'Dd', 'De', 'Dh', 'Di',
# 'Dj', 'Dl', 'Dn', 'Do', 'Dr', 'Ds', 'Dt', 'Du', 'Dw', 'Dx', 'Dz', 'Dà', 'E', 'EA', 'EB', 'EC', 'ED', 'EE', 'EF', 'EG',
# 'EH', 'EI', 'EJ', 'EK', 'EL', 'EM', 'EN', 'EO', 'EP', 'EQ', 'ER', 'ES', 'ET', 'EU', 'EV', 'EW', 'EX', 'EY', 'EZ', 'Ea',
# 'Ec', 'Ed', 'Ee', 'Eg', 'Eh', 'Ei', 'Ej', 'El', 'Em', 'En', 'Eo', 'Ep', 'Er', 'Es', 'Et', 'Eu', 'Ev', 'Ew', 'Ex', 'Ey',
# 'Ez', 'F', 'FA', 'FB', 'FC', 'FD', 'FE', 'FF', 'FG', 'FH', 'FI', 'FJ', 'FK', 'FL', 'FM', 'FN', 'FO', 'FP', 'FR', 'FS',
# 'FT', 'FU', 'FW', 'FX', 'Fa', 'Fb', 'Fe', 'Fi', 'Fk', 'Fl', 'Fm', 'Fn', 'Fo', 'Fr', 'Fs', 'Ft', 'Fu', 'G', 'GA', 'GB',
# 'GC', 'GD', 'GE', 'GF', 'GG', 'GH', 'GI', 'GJ', 'GK', 'GL', 'GM', 'GN', 'GO', 'GP', 'GQ', 'GR', 'GS', 'GT', 'GV', 'GW',
# 'Ga', 'Gd', 'Ge', 'Gf', 'Gi', 'Gj', 'Gm', 'Go', 'Gp', 'Gr', 'Gs', 'Gu', 'Gy', 'H', 'HA', 'HB', 'HC', 'HD','HE', 'HF', 'HG', 'HH', 'HI', 'HJ', 'HK', 'HM', HO', 'HP', 'HQ', 'HR', 'HS', 'HT', 'HV', 'HW', 'HX', 'Ha', 'Hb', 'He',
# 'Hh', 'Hi', 'Hj', 'Hk', 'Hm', 'Ho', 'Hr', 'Hs', 'Hu', 'Hv', 'Hw', 'Hy', 'Hà', 'Hù', 'I', 'IA', 'IB', 'IC', 'ID', 'IE',
# 'IF', 'IG', 'II', 'IL', 'IM', 'IN', 'IO', 'IP', 'IQ', 'IS', 'IT', 'IU', 'IV', 'IW', 'IX', 'IZ', 'Ic', 'Id', 'Ie', 'If',
# 'Ig', 'Ii', 'Ik', 'Il', 'Im', 'In', 'Io', 'Ip', 'Ir', 'Is', 'It', 'Iv', 'Iw', 'Ix', 'Iz', 'J', 'JA', 'JB', 'JC', 'JD',
# 'JF', 'JG', 'JH', 'JI', 'JJ', 'JK', 'JL', 'JM', 'JO', 'JP', 'JQ', 'JR', 'JS', 'JT', 'JV', 'JW', 'JY', 'JZ', 'Ja', Jc',
# 'Jd', 'Je', 'Ji', 'Jk', 'Jo', 'Jp', 'Jr', 'Js', 'Jt', 'Ju', 'Já', 'K', 'KA', 'KB', 'KC', 'KD', 'KF', 'KG', 'KH', 'KJ',
# 'KK', 'KL', 'KM', 'KN', 'KO', 'KP', 'KR', 'KS', 'KT', 'KU', 'KW', 'KY', 'Ka', 'Ke', 'Kg', 'Ki', 'Kj', 'Ko', 'Ks', 'Ku',
# 'Ky', 'KÀ', 'KÁ', 'Kà', 'Ká', 'Kā', 'L', 'LA', 'LB', 'LC', 'LD', 'LE', 'LF', 'LG', 'LH', 'LI', 'LJ', 'LK', 'LL', 'LM',
# 'LN', 'LO', 'LP', 'LQ', 'LR', 'LS', 'LT', 'LU', 'LV', 'LX', 'LY', 'LZ', 'La', 'Lb', 'Le', 'Lg', 'Li', 'Ll', 'Lm', 'Ln',
# 'Lo', 'Ls', 'Lt', 'Lu', 'Lv', 'Ly', 'Là', 'Lá', 'Lé', 'M', 'MA', 'MB', 'MC', 'MD', 'ME', 'MF', 'MG', 'MH', 'MI', 'MJ',
# 'MK', 'ML', 'MM', 'MN', 'MO', 'MP', 'MR', 'MS', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', 'Ma', 'Mb', 'Mc', 'Md', 'Me',
# 'Mg', 'Mh', 'Mi', 'Mj', 'Mm', 'Mn', 'Mo', 'Mp', 'Mr', 'Ms', 'Mt', 'Mu', 'Mw', 'Mx', 'My', 'Mì', 'N', 'NA', 'NB', 'NC',
# 'ND', 'NE', 'NF', 'NG', 'NH', 'NI', 'NJ', 'NL', 'NM', 'NN', 'NO', 'NP', 'NS', 'NT', 'NU', 'NV', 'NW', 'NY', 'NZ', 'Na',
# 'Nd', 'Ne', 'Ng', 'Nh', 'Ni', 'Nj', 'Nm', 'No', 'Np', 'Ns', 'Nu', 'Nv', 'Ny', 'O', 'OB', 'OC', 'OD', 'OE', 'OF', 'OG',
# 'OH', 'OI', 'OJ', 'OK', 'OL', 'OM', 'ON', 'OO', 'OP', 'OR', 'OS', 'OT', 'OU', 'OV', 'OW', 'OX', 'OY', 'OZ', 'Ob', 'Oc',
# 'Od', 'Of', 'Og', 'Oh', 'Oi', 'Oj', 'Ok', 'Ol', 'Om', 'On', 'Oo', 'Op', 'Or', 'Os', 'Ot', 'Ou', 'Ow', 'Ox', 'Oy', 'Oz',
# 'Oí', 'P', 'PA', 'PB', 'PC', 'PD', 'PE', 'PF', 'PG', 'PH', 'PI', 'PJ', 'PK', 'PL', 'PM', 'PN', 'PO', 'PP', 'PR', 'PS',
# 'PT', 'PU', 'PV', 'PW', 'PX', 'Pa', 'Pb', 'Pc', 'Pd', 'Pe', 'Pf', 'Ph', 'Pi', 'Pj', 'Pm', 'Po', 'Pr', 'Ps', 'Pt', 'Pu',
# 'Pà', 'På', 'Pó', 'Q', 'QA', 'QB', 'QC', 'QF', 'QI', 'QK', 'QQ', 'QR', 'QT', 'Qc', 'Qe', 'Qi', 'Qs', 'Qu', 'R', 'RA',
# 'RB', 'RC', 'RD', 'RE', 'RF', 'RH', 'RI', 'RJ', 'RK', 'RL', 'RM', 'RN', 'RO', 'RP', 'RR', 'RS', 'RT', 'RU', 'RV', 'RW',
# 'RX', 'Ra', 'Rd', 'Re', 'Ri', 'Rj', 'Rm', 'Ro', 'Rs', 'Rt', 'Ru', 'Rv', 'Rx', 'Ry', 'Rá', 'Rí', 'S', 'SA', 'SB', 'SC',
# 'SD', 'SE', 'SF', 'SG', 'SH', 'SI', 'SJ', 'SK', 'SL', 'SM', 'SN', 'SO', 'SP', 'SR', 'SS', 'ST', 'SU', 'SV', 'SW', 'SX',
# 'SY', 'Sa', 'Sc', 'Sd', 'Se', 'Sf', 'Sh', 'Si', 'Sk', 'Sm', 'Sn', 'So', 'Sp', 'Sq', 'Sr', 'Ss', 'St', 'Su', 'Sw', 'Sy',
# 'SÍ', 'Så', 'Sí', 'Só', 'Sô', 'T', 'TA', 'TB', 'TC', 'TD', 'TE', 'TF', 'TG', 'TH', 'TI', 'TJ', 'TK', 'TL', 'TM', 'TN',
# 'TO', 'TP', 'TR', 'TS', 'TT', 'TU', 'TV', 'TW', 'TX', 'TY', 'TZ', 'Ta', 'Te', 'Tf', 'Th', 'Ti', 'Tj', 'Tk', 'Tl', 'Tm',
# 'Tn', 'To', 'Tp', 'Tr', 'Ts', 'Tt', 'Tu', 'Tv', 'Tx', 'Ty', 'Tz', 'Tô', 'Tŷ', 'U', 'UA', 'UC', 'UG', 'UH', 'UI', 'UJ',
# 'UK', 'UL', 'UM', 'UN', 'UO', 'UP', 'UR', 'US', 'UT', 'UV', 'UW', 'Ua', 'Ud', 'Ug', 'Uh', 'Uk', 'Um', 'Un', 'Up', 'Ur',
# 'Us', 'Ut', 'Uw', 'Uy', 'V', 'VA', 'VB', 'VC', 'VD', 'VE', 'VG', 'VH', 'VI', 'VJ', 'VK', 'VM', 'VN', 'VP', 'VR', 'VS',
# 'VT', 'VU', 'VV', 'VW', 'VX', 'VZ', 'Va', 'Ve', 'Vi', 'Vm', 'Vn', 'Vs', 'Vt', 'Vu', 'Vy', 'Và', 'Vá', 'W', 'WA', 'WB',
# 'WC', 'WD', 'WE', 'WF', 'WG', 'WI', 'WJ', 'WK', 'WM', 'WO', 'WP', 'WR', 'WS', 'WT', 'WU', 'WV', 'WW', 'WY', 'Wa', 'We',
# 'Wi', 'Wn', 'Wo', 'Wr', 'Ws', 'Wu', 'Ww', 'X', 'XD', 'XI', 'XL', 'XM', 'XO', 'XP', 'XR', 'XS', 'XT', 'XV', 'XX', 'XY',
# 'Xa', 'Xe', 'Xi', 'Xk', 'Xl', 'Xo', 'Xs', 'Xu', 'Xx', 'Y', 'YA', 'YB', 'YC', 'YD', 'YE', 'YG', 'YH', 'YI', 'YK', 'YL',
# 'YM', 'YO', 'YS', 'YT', 'YU', 'YY', 'Ya', 'Yb', 'Ye', 'Yi', 'Yk', 'Yo', 'Yr', 'Ys', 'Yu', 'Z', 'ZA', 'ZB', 'ZD', 'ZE',
# 'ZI', 'ZK', 'ZZ', 'Za', 'Ze', 'Zi', 'Zo', 'Zu', 'Zy', 'aD', 'aI', 'aM', 'aS', 'aT', 'aa', 'aaa', 'aaaa', 'aaaaa',
# 'aaaaaa', 'aaaaaaaaa', 'aaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaand', 'aaaaaaaaaaaaaauthent',
# 'aaaaaaaaaaaaaawwwwwwwwwwwwwwwwwwwwww', 'aaaaaaaaaaaah', 'aaaaaaaaaaaahhhhhh', 'aaaaaaaaaaah', 'aaaaaaaaaaand',
# 'aaaaaaaaaand', 'aaaaaaaaawesom', 'aaaaaaaaawww', 'aaaaaaaah', 'aaaaaaaahhhhhhhhh', 'aaaaaaaamaz', 'aaaaaaaand',
# 'aaaaaaaargh','x']
# las_vegas_data['tokenized_text']=las_vegas_data['tokenized_text'].apply(lambda x: [item for item in x if item not in word_list])
# print("Time to load data: {} seconds".format(time.time() - start))

In [0]:
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= GOOGLE_APPLICATION_CREDENTIALS='/content/drive/My Drive/yelp-language-detection-ab4d472516d3.json'
# from google.cloud import translate
# translate_client = translate.Client()
# las_vegas_data['language']=las_vegas_data['text'].apply(translate_client.detect_language)